In [1]:
# -*- coding: utf-8 -*-
import yaml
import data
from data.read_data_temp import select_data,TrainData,file_split,DataByFile,sample_negative
from data.data_split import kfsplit
from data.datasets import prepare_loaders
from utils.wandb_init import get_run,wandb_utils
from utils.train import get_score,train_set,get_score_list
from utils.trivial import get_logger
from model.head import OriginModel
from model.trainer import train,predict
import torch
from transformers import logging

import wandb

import numpy as np
import pandas as pd 
import os
import torch

from sklearn.metrics import precision_score,recall_score,f1_score
import gc 
gc.enable()
from utils.trivial import set_seed

import warnings
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)


In [2]:
yaml_path="./config/train.yaml"
CONFIG,Logger,DEVICE=train_set(yaml_path,experimentName=None,upload=False,filename="./test/logs",is_notebook=True)

CONFIG["num_hidden_layers"]=12
CONFIG["DATA_NUM"]=50000
CONFIG["EPOCHS"]=1
CONFIG["DEBUG_MODEL"]=False
CONFIG["run_db"]=False


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: adreambear. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/v-mxiong/.netrc


In [3]:
    

CONFIG["INPUT_DIR"]="/vc_data/users/v-mxiong/qc/QueryTitle"
with open(os.path.join(CONFIG["INPUT_DIR"],"Schema"),"r") as f:
    line=f.readlines()[0].strip()
    column_name=line.split("\t")
    CONFIG["Logger"].info(f"column_name :{column_name}")

# trainDataObj=DataByFile(CONFIG,"train")
# trainData=trainDataObj.get_data()
# CONFIG["Logger"].info("-"*10+"trainData label's value_counts"+"-"*10)
# CONFIG["Logger"].info(trainData["label"].value_counts())
# CONFIG["Logger"].info("-"*10+"trainData fold's value_counts"+"-"*10)
# CONFIG["Logger"].info(trainData["fold"].value_counts())


column_name :['Query', 'label', 'dsat']


In [8]:
from utils.trivial import timer

@timer
def read_data_file(input_file,is_debug,data_num,head=["Query","label"]):
    with open(input_file,"r",encoding="UTF-8") as input_object:
        if is_debug==True:
            input_lines=[]
            for _ in range(data_num):
                input_lines.append(input_object.readline())
        else:
            input_lines=input_object.readlines()
    input_lines=[ line.strip().split("\t") for line in input_lines]
    # may return wrong number 
    input_lines=[ line for line in input_lines if len(line)==len(head)]
    # input_lines=[ [line[0].split("$$")[0],line[1]] for line in input_lines if len(line)==len(head)]
    
    # url+title
    # input_lines=[ [" ".join(line[0].split("$$")[1:3]),line[1]] for line in input_lines if len(line)==len(head)]
    
    input_df=pd.DataFrame(input_lines,columns=head)
    return input_df

In [14]:
import threading
import sys
import queue
 
 
class Reader(threading.Thread):
    def __init__(self, thread_id,column_num):
        super(Reader, self).__init__()
        self.thread_id = thread_id
        self.column_num=column_num
        self.lines=[]
    def run(self):
        global q
 
        input_lines = q.get()
        input_lines=[ line.strip().split("\t") for line in input_lines]
        input_lines=[ line for line in input_lines if len(line)==self.column_num]
        self.lines=input_lines
 

def wc_count(file_name):
    import subprocess
    out = subprocess.getoutput("wc -l %s" % file_name)
    return int(out.split()[0])

class Partition(object):
    def __init__(self, file_name, thread_num):
        self.file_name = file_name
        self.block_num = thread_num
 
    #按照线程数对文件进行分块并存进queue中
    
    @timer 
    def part_and_queue(self):
        pos_list = []
        #文件总行数
        file_size = wc_count(self.file_name)
        #按照线程数分成对应块的大小
        block_size = file_size / self.block_num
        start_pos = 0
        global q
 
        for i in range(self.block_num):
            if i == self.block_num - 1:
                end_pos = file_size - 1
                pos_list.append((start_pos, end_pos))
                break
            end_pos = start_pos + block_size - 1
            if end_pos >= file_size:
                end_pos = file_size - 1
            if start_pos >= file_size:
                break
            pos_list.append((start_pos, end_pos))
            start_pos = end_pos + 1
 
        #读取每块内容存进queue中
        fd = open(self.file_name, 'r')
        for pos_tu in pos_list:
            temp_text = []
            start = pos_tu[0]
            end = pos_tu[1]
 
            while start <= end:
                text = fd.readline().strip('\n')
                temp_text.append(text)
                start = start + 1
 
            q.put(temp_text)
        fd.close()
 
 

    

In [18]:
input_file=os.path.join(CONFIG["INPUT_DIR"],"NegativeTest")
df1=read_data_file(input_file,CONFIG["DEBUG_MODEL"],CONFIG["DATA_NUM"],head=column_name)

read_data_file cost time: 193.264 s


In [19]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56243689 entries, 0 to 56243688
Data columns (total 3 columns):
Query    object
label    object
dsat     object
dtypes: object(3)
memory usage: 1.3+ GB


In [20]:
del df1

In [27]:
file_name =input_file
 
#线程数量可配
thread_num = 32
q = queue.Queue()
p = Partition(file_name, thread_num)
t = []
p.part_and_queue()



part_and_queue cost time: 72.663 s


In [28]:
@timer
def test1():
    for i in range(thread_num):
        t.append(Reader(i,column_num=len(column_name)))
    for i in range(thread_num):
        t[i].start()
    for i in range(thread_num):
        t[i].join()
        
    input_lines=[]
    for i in range(thread_num):
        input_lines.extend(t[i].lines)

    input_df=pd.DataFrame(input_lines,columns=column_name)
    return input_df
    
input_df=test1()

test1 cost time: 170.273 s


In [29]:
input_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56243663 entries, 0 to 56243662
Data columns (total 3 columns):
Query    object
label    object
dsat     object
dtypes: object(3)
memory usage: 1.3+ GB


In [24]:
del input_df

In [35]:
def safe_readline(f):
    pos = f.tell()
    while True:
        try:
            return f.readline()
        except UnicodeDecodeError:
            pos -= 1
            f.seek(pos)
            
            
def async_kd_tokenizer(filename, worker_id, num_workers,column_num):
    with open(filename, 'r') as f:
        size = os.fstat(f.fileno()).st_size  # 指针操作，所以无视文件大小
        print(f'size {size}')
        chunk_size = size // num_workers
        offset = worker_id * chunk_size
        end = offset + chunk_size
        f.seek(offset)
        print(f'offset {offset}')
        if offset > 0:
            safe_readline(f)    # drop first incomplete line
        lines = []
        line = f.readline()
        while line:
            line=line.strip().split("\t")
            if len(line)==column_num:
                lines.append(line)
            if f.tell() > end:
                break
            line = f.readline()
        return lines


In [36]:
import threading


class FileHandlerThread(threading.Thread):

    def __init__(self, func, args):
        super(FileHandlerThread, self).__init__()
        self.args = args
        self.func = func
        
    def run(self):
        self.result = self.func(*self.args)
    
    def get_result(self):
        try:
            return self.result
        except Exception:
            return None

    
def encode_file_thread(path, workers=4):
    assert os.path.exists(path)
    results = []
    workers_thread = []
    for i in range(workers):
        w = FileHandlerThread(async_kd_tokenizer, args=(path, i, workers,len(column_name)))
        workers_thread.append(w)
        w.start()
    for w in workers_thread:
        w.join()
    for w in workers_thread:
        result = w.get_result() 
        results.extend(result)
    return results
	


In [37]:
file_name =input_file

results_th = encode_file_thread(file_name, workers=8)


size 1085609355
offset 135701169
size 1085609355
offset 0
size 1085609355
offset 271402338
size 1085609355
offset 407103507
size 1085609355
offset 542804676
size 1085609355
offset 678505845
size 1085609355
offset 814207014
size 1085609355
offset 949908183


KeyboardInterrupt: 